In [5]:
import functions as fc
import pandas as pd


In [ ]:
fc.download_tgz_raw()

In [ ]:
fc.import_tgz("2018", "./data2018")

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
def transform_rcs(year):
    list_df = fc.import_all_files("data" + year + "/", "*.zip"
    print(len(list_df))
    li = []
    for i in range(len(list_df)):
        temp = pd.concat(list_df[i])
        li.append(temp)
    df_final = pd.concat(li)
    print(df_final.shape)

In [2]:
list_df = fc.import_all_files("data2019/", "*.zip")

/home/coder/work/data_inpi/code/functions.py:41: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:41: DtypeWarning: Columns (17,32,35,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:41: DtypeWarning: Columns (7,14,25,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:41: DtypeWarning: Columns (7,14,15,17,24,25,27,28,29,30,31,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:41: DtypeWarning: Columns (39,40,42) have mixed types. Specify dtype option on import or set low_memory=Fa

In [3]:
len(list_df)

668

In [6]:
li = []
for i in range(len(list_df)):
    temp = pd.concat(list_df[i])
    li.append(temp)

In [7]:
df_final = pd.concat(li)

In [8]:
df_final.shape

(5888180, 118)

In [9]:
df_final.head()

,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,Sans_Activité,...,Enseigne,Activité_Ambulante,Activité_Saisonnière,Activité_Non_Sédentaire,Date_Début_Activité,Activité,Origine_Fonds,Origine_Fonds_Info,Type_Exploitation,ID_Etablissement
0,101,Bourg-en-Bresse,2019B00890,305148884,S,2019-05-24,2019-05-24,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101,Bourg-en-Bresse,1976B40029,308529668,P,1976-11-26,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101,Bourg-en-Bresse,2002D00732,309729879,P,2002-10-30,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101,Bourg-en-Bresse,2011B01046,317272243,P,2011-09-06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101,Bourg-en-Bresse,1997B00138,319026720,P,1997-03-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/inpi_2018.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    dfs.to_csv(file_out)

In [ ]:
fc.export_2_minio("2019", df_final)

In [10]:

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/inpi_2019.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final.to_csv(file_out)

In [ ]:
import s3fs
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/inpi_2020.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final.to_csv(file_out)